In [1]:
import pandas as pd
import numpy as np
import csv as csv
from datetime import *
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import random
from scipy.stats import mode
from collections import Counter  
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold, GroupKFold
%matplotlib inline

plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['text.color'] = 'white'

In [2]:
# 导入数据
df = pd.read_table('../data/oppo_data_ronud2_20181107/data_train.txt', 
                   names= ['prefix','query_prediction','title','tag','label'], header=None, na_values='', keep_default_na=False, encoding='utf-8', quoting=3)
print(df.count())
print(df.head())
validDf = pd.read_table('../data/oppo_data_ronud2_20181107/data_vali.txt', 
                        names = ['prefix','query_prediction','title','tag','label'], header=None, na_values='', keep_default_na=False, encoding='utf-8', quoting=3)
# predictDf = pd.read_table('../data/oppo_data_ronud2_20181107/data_test.txt', 
#                         names = ['prefix','query_prediction','title','tag','label'], header=None, na_values='', keep_default_na=False, encoding='utf-8', quoting=3)
testDf = pd.read_table('../data/oppo_data_ronud2_20181107/data_testB.txt', 
                        names = ['prefix','query_prediction','title','tag','label'], header=None, na_values='', keep_default_na=False, encoding='utf-8', quoting=3)

print(validDf.info())
# print(predictDf.info())
print(testDf.info())

prefix              2000000
query_prediction    1946144
title               2000000
tag                 2000000
label               2000000
dtype: int64
  prefix                                   query_prediction  \
0      羁  {"羁": "0.136", "羁绊怎么读": "0.027", "羁绊": "0.141"...   
1   六合                                                NaN   
2    红岩金  {"红岩金刚后八轮": "0.024", "红岩金刚二手车": "0.044", "红岩金刚...   
3   高德地图  {"高德地图": "0.576", "高德地图免费导航": "0.117", "高德地图免费...   
4  12306  {"12306官网": "0.081", "12306app": "0.007", "123...   

                title tag  label  
0                   羁  百科      1  
1          六间房秀场-视频直播  应用      0  
2  红岩金刚仪表盘上的灯各代表什么意思？  知道      0  
3                高德地图  应用      1  
4         12306铁路订票官网  网站      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
prefix              50000 non-null object
query_prediction    48739 non-null object
title               50000 non-null object
tag                 50000 

In [3]:
def formatQuery(df):
    '''
    格式化预测词字段
    '''
    tempDf = df[['prefix','query_prediction']].drop_duplicates(subset='prefix')
    def format(x):
        x = eval(x)
        x = {k:float(v) for k,v in x.items()}
        return x
    tempDf['query_prediction'] = tempDf['query_prediction'].dropna().map(lambda x: format(x))
    tempDf['query_predict_num'] = tempDf['query_prediction'].dropna().map(lambda x: len(x))
    tempDf.loc[tempDf.query_predict_num==0, 'query_prediction'] = np.nan
    tempDf['query_predict_num'].fillna(0, inplace=True)
    tempDf['query_word'] = tempDf['query_prediction'].dropna().map(lambda x: list(x.keys()))
    tempDf['query_ratio'] = tempDf['query_prediction'].dropna().map(lambda x: list(x.values()))
    df = df.drop(['query_prediction'], axis=1).merge(tempDf, how='left', on=['prefix'])
    return df

df = formatQuery(df)
validDf = formatQuery(validDf)
# predictDf = formatQuery(predictDf)
testDf = formatQuery(testDf)

In [4]:
# 统计特征函数
def addCrossColNunique(df, statDf, col1, col2, alias=None):
    '''
    统计每个col1中col2的种类数
    '''
    if alias is None:
        alias = '%s_%s' % (col1, col2)
    if '%s_nunique'%alias in statDf.columns:
        tempDf = statDf[[col1, '%s_nunique'%alias]].drop_duplicates(subset=col1)
        df = df.merge(tempDf, how='left', on=col1)
    else:
        tempDf = statDf.groupby(col1)[col2].nunique().to_frame()
        tempDf.columns = ['%s_nunique'%alias]
        df = df.merge(tempDf.reset_index(), how='left', on=col1)
    return df

def addLabelFea(df, statDf, colArr, alias=None):
    '''
    统计列的标签独立数和标签点击率
    '''
    if len(np.array(colArr).shape) == 0:
        colArr = [colArr]
    if alias is None:
        alias = '_'.join(np.array(colArr).astype(str))
    if statDf[colArr].count().min() == 0:
        df['%s_label_len'%alias] = df['%s_label_sum'%alias] = 0
        df['%s_label_ratio'%alias] = df['%s_label_ratio2'%alias] = np.nan
        return df
    if '%s_label_ratio'%alias in statDf.columns:
        tempDf = statDf[colArr + ['%s_label_len'%alias, '%s_label_sum'%alias, '%s_label_ratio'%alias]].drop_duplicates(subset=colArr)
        df = df.merge(tempDf, how='left', on=colArr)
        df.fillna({'%s_label_len'%alias: 0, '%s_label_sum'%alias: 0}, inplace=True)
    else:
        tempDf = statDf.groupby(colArr)['label'].agg([len, 'sum'])
        tempDf['ratio'] = tempDf['sum'] / tempDf['len']
#         tempDf.loc[:,['len','sum']] /= tempDf[['len','sum']].sum()
        tempDf.columns = ['%s_label_%s'%(alias,x) for x in tempDf.columns]
        df = df.merge(tempDf.reset_index(), 'left', on=colArr)
        df.fillna({'%s_label_len'%alias: 0, '%s_label_sum'%alias: 0}, inplace=True)
    return df

def addNewValFea(df, statDf, cols):
    '''
    判断字段值是否在统计表中出现过
    '''
    if len(np.array(cols).shape) == 0:
        cols = [cols]
    for col in cols:
        if statDf[col].count() == 0:
            df['%s_newVal'%col] = 1
        else:
            tempDf = df[[col]].drop_duplicates()
            existList = np.intersect1d(tempDf[col].dropna(),statDf[col].dropna())
            tempDf['%s_newVal'%col] = (~tempDf[col].isin(existList)).astype(int)
            df = df.merge(tempDf, 'left', on=[col])
    return df


In [5]:
# 全局特征跟五折特征汇总函数
def addHisFeas(df, statDf):
    '''
    添加历史统计类特征
    '''
    startTime = datetime.now()
    df = addGlobalFeas(df, statDf)
    df = addNewValFea(df, statDf, ['prefix','title','tag'])
    # 统计点击率特征
    colList = ['prefix','title','tag',['prefix','title'],['prefix','tag'],['title','tag']]
    for col in colList:
        df = addLabelFea(df, statDf, col)
    return df

def addGlobalFeas(df, statDf=None):
    '''
    添加全局特征
    '''
    if statDf is None:
        statDf = df
    # 统计交叉维度独立数
    crossColList = [
        ['prefix','title'],
        ['title','prefix'],
        ['prefix','tag'],
        ['title','tag'],
    ]
    for c1,c2 in crossColList:
        df = addCrossColNunique(df, statDf, c1, c2)
    return df


In [6]:
# 多折交叉统计函数
def addCvHisFea(df, nFold=5, random_state=0):
    '''
    多折交叉添加历史统计特征
    '''
    kf = StratifiedKFold(n_splits=nFold, random_state=random_state, shuffle=True)
    dfList = []
    for i, (statIdx, taskIdx) in enumerate(kf.split(df.values, df['label'].values)):
        tempDf = addHisFeas(df.iloc[taskIdx], df.iloc[statIdx])
        dfList.append(tempDf)
    df = pd.concat(dfList, ignore_index=True)
    return df

def addCvHisFea2(df, nFold=5, random_state=0):
    '''
    多折交叉添加历史统计特征
    '''
    kf = KFold(n_splits=nFold)
    dfList = []
    for i, (statIdx, taskIdx) in enumerate(kf.split(df.values, df['label'].values)):
        tempDf = addHisFeas(df.iloc[taskIdx], df.iloc[statIdx])
        dfList.append(tempDf)
    df = pd.concat(dfList, ignore_index=True)
    return df

In [7]:
df['flag'] = 0
df = df.reset_index().rename(columns={'index':'instance_id'})
validDf['flag'] = 1
validDf = validDf.reset_index().rename(columns={'index':'instance_id'})
# predictDf['flag'] = -1
# predictDf = predictDf.reset_index().rename(columns={'index':'instance_id'})
testDf['flag'] = -2
testDf = testDf.reset_index().rename(columns={'index':'instance_id'})

statDf = pd.concat([df,validDf])
# statDf = addGlobalFeas(statDf, df)
# df = addGlobalFeas(df, df)
# statDf = addGlobalFeas(statDf, statDf)
# validDf = addGlobalFeas(validDf, df)
# predictDf = addGlobalFeas(predictDf, statDf)
# testDf = addGlobalFeas(testDf, statDf)

# predictDf = addHisFeas(predictDf, statDf)
testDf = addHisFeas(testDf, statDf)
validDf = addHisFeas(validDf, df)
newDf = addCvHisFea2(df, nFold=5)
# oldDf = addCvHisFea(df, nFold=5)
statDf = addCvHisFea2(statDf, nFold=5)
# df3 = addCvHisFea(df, nFold=3)
# df10 = addCvHisFea2(df, nFold=10)
# df1 = addCvHisFea(df, nFold=5)

In [9]:
# 输出重合情况
def printInter(df):
    for x in ['prefix','title']:
        print('%s的新样本数:'%x, df['%s_newVal'%x].sum(), '%s的新样本率:'%x, df['%s_newVal'%x].sum() / df.shape[0])
        tempDf = df.drop_duplicates([x])
        print('%s的新种类数:'%x, tempDf['%s_newVal'%x].sum(), '新种类率:', tempDf['%s_newVal'%x].sum() / tempDf.shape[0])
    print('只缺失prefix的样本数:', df[(df.prefix_newVal==1)&(df.title_newVal==0)].shape[0])
    print('只缺失title的样本数:', df[(df.prefix_newVal==0)&(df.title_newVal==1)].shape[0])
    print('同时缺失prefix和title的样本数:', df[(df.prefix_newVal==1)&(df.title_newVal==1)].shape[0])

In [11]:
print('--------5折交叉分布：---------')
printInter(df)
print('--------验证集分布--------')
printInter(validDf)
print('--------测试集分布（训练+验证统计）--------')
printInter(predictDf)
print('--------测试集B分布（训练+验证统计）--------')
printInter(testDf)

--------5折交叉分布：---------
prefix的新样本数: 940520 prefix的新样本率: 0.47026
prefix的新种类数: 143703 新种类率: 0.6772980284770304
title的新样本数: 533323 title的新样本率: 0.2666615
title的新种类数: 266534 新种类率: 0.7595270730449304
只缺失prefix的样本数: 493298
只缺失title的样本数: 86101
同时缺失prefix和title的样本数: 447222
--------验证集分布--------
prefix的新样本数: 18824 prefix的新样本率: 0.37648
prefix的新种类数: 5601 新种类率: 0.25818198580252605
title的新样本数: 8832 title的新样本率: 0.17664
title的新种类数: 6130 新种类率: 0.24829876863253403
只缺失prefix的样本数: 11402
只缺失title的样本数: 1410
同时缺失prefix和title的样本数: 7422
--------测试集分布（训练+验证统计）--------
prefix的新样本数: 82246 prefix的新样本率: 0.41123
prefix的新种类数: 23262 新种类率: 0.3577778461349165
title的新样本数: 40448 title的新样本率: 0.20224
title的新种类数: 26734 新种类率: 0.3430734680782804
只缺失prefix的样本数: 46942
只缺失title的样本数: 5144
同时缺失prefix和title的样本数: 35304
--------测试集B分布（训练+验证统计）--------
prefix的新样本数: 82406 prefix的新样本率: 0.41203
prefix的新种类数: 17696 新种类率: 0.29737677920244676
title的新样本数: 32491 title的新样本率: 0.162455
title的新种类数: 21605 新种类率: 0.3022692931893223
只缺失prefix的样本数: 55

+ 训练集跟初赛训练集分布类似，验证集和测试集均跟初赛验证集的分布类似
*以下为随机五折交叉的分析，随机交叉5这的prefix新样本率是0.08*
+ 测试集新数据样本率暴增，不再与5折的新样本率接近
+ 训练集中新prefix的平均样本量为1.1，符合样本抽样时小概率样本缺失的规律。但验证集和测试集中的新prefix平均样本量均超过3，这点不太正常
*以下为对比了不随机五折交叉的分析*
+ 不随机五折出来的新样本率更接近验证集和测试集的分布，结合隔壁数据分析上

In [10]:
print('--------3折交叉分布：---------')
printInter(df3)
print('--------5折交叉分布：---------')
printInter(df)
print('--------10折交叉分布：---------')
printInter(df10)

--------3折交叉分布：---------
prefix的新样本数: 119719 prefix的新样本率: 0.0598595
prefix的新种类数: 94171 新种类率: 0.443844823279336
title的新样本数: 258521 title的新样本率: 0.1292605
title的新种类数: 234954 新种类率: 0.6695353085167317
只缺失prefix的样本数: 22458
只缺失title的样本数: 161260
同时缺失prefix和title的样本数: 97261
--------5折交叉分布：---------
prefix的新样本数: 97661 prefix的新样本率: 0.0488305
prefix的新种类数: 84481 新种类率: 0.3981741142757493
title的新样本数: 238352 title的新样本率: 0.119176
title的新种类数: 226218 新种类率: 0.6446408165940483
只缺失prefix的样本数: 18838
只缺失title的样本数: 159529
同时缺失prefix和title的样本数: 78823
--------10折交叉分布：---------
prefix的新样本数: 84180 prefix的新样本率: 0.04209
prefix的新种类数: 78194 新种类率: 0.36854235498725085
title的新样本数: 225896 title的新样本率: 0.112948
title的新种类数: 220428 新种类率: 0.6281413765491378
只缺失prefix的样本数: 16681
只缺失title的样本数: 158397
同时缺失prefix和title的样本数: 67499


In [86]:
# 统计不同数据集文章标签的概率分布
print('不同tag的样本量占总样本的比例')
tempDf = pd.concat([df, validDf, predictDf])
tempDf = pd.pivot_table(tempDf, index=['tag'], columns=['flag'], values='label', aggfunc=len)
print(tempDf / tempDf.sum())

不同tag的样本量占总样本的比例
flag        -1         0        1
tag                              
健康    0.058870  0.057281  0.05456
商品    0.002540  0.002476  0.00208
应用    0.182830  0.190793  0.20536
影视    0.020470  0.018881  0.02824
快应用   0.006170  0.007522  0.00932
旅游    0.001840  0.001404  0.00142
景点    0.006280  0.007629  0.00580
歌手    0.007135  0.006635  0.00608
汽车    0.012855  0.012710  0.01060
游戏    0.016690  0.012495  0.00964
火车    0.000560  0.000553  0.00048
百科    0.349390  0.341603  0.33100
知道    0.043965  0.044131  0.04244
经验    0.046435  0.045887  0.04428
网站    0.056715  0.061348  0.06888
网页    0.086885  0.086185  0.08300
航班    0.000335  0.000296  0.00040
菜谱    0.041300  0.042691  0.04058
酒店    0.000160  0.000178  0.00018
阅读    0.028645  0.029589  0.02728
音乐    0.029930  0.029711  0.02838


**在标签抽样维度上，三个数据集没有差别，但跟初赛数据集不一致**

In [102]:
# 统计不同数据集，prefix对应的title种类数的规律
printDf = pd.DataFrame()
for tempDf in [df, validDf, predictDf]:
    tempDf = tempDf.drop_duplicates(['prefix'])
#     tempDf = tempDf.groupby('prefix')['title'].agg('nunique')
    tempDf = tempDf['prefix_title_nunique'].value_counts() / tempDf.shape[0]
    printDf = pd.concat([printDf, tempDf], axis=1)
printDf.columns = ['train','valid','test']
print(printDf)

       train     valid      test
1   0.361572  0.239098  0.300824
2   0.349162  0.202176  0.246824
3   0.186114  0.218309  0.224322
4   0.057755  0.139439  0.110739
5   0.026182  0.096110  0.062521
6   0.011236  0.053471  0.030668
7   0.004680  0.027104  0.013735
8   0.002135  0.015027  0.006629
9   0.000716  0.005624  0.002322
10  0.000278  0.002213  0.000861
11  0.000099  0.000784  0.000323
12  0.000057  0.000507  0.000185
13  0.000009  0.000092  0.000031
14  0.000005  0.000046  0.000015


**训练集中3的界限依旧明显，但比初赛数据集低了不少。验证集和测试集中，只有一两种title的搜索词比例下降，超过3种的比例有所提高，断层不再明显**

In [72]:
# 统计不同数据集，title对应的prefix种类数的规律
printDf = pd.DataFrame()
for tempDf in [df, validDf, predictDf]:
    tempDf = tempDf.drop_duplicates(['prefix','title'])
    tempDf = tempDf.groupby('title')['prefix'].agg('nunique')
    tempDf = tempDf.value_counts() / tempDf.shape[0]
    printDf = pd.concat([printDf, tempDf], axis=1)
printDf.columns = ['train','valid','test']
print(printDf)

        train     valid      test
1    0.877269  0.930695  0.904909
2    0.094124  0.049336  0.073250
3    0.018172  0.009033  0.012101
4    0.004864  0.003564  0.003503
5    0.001889  0.001782  0.001476
6    0.000883  0.001134  0.000937
7    0.000581  0.000851  0.000500
8    0.000333  0.000527  0.000411
9    0.000219  0.000567  0.000295
10   0.000177  0.000365  0.000282
11   0.000134  0.000122  0.000154
12   0.000100  0.000324  0.000167
13   0.000123  0.000122  0.000141
14   0.000060  0.000122  0.000167
15   0.000034  0.000122  0.000141
16   0.000060  0.000324  0.000077
17   0.000040  0.000081  0.000128
18   0.000046  0.000041  0.000051
19   0.000017  0.000122  0.000103
20   0.000028  0.000162  0.000090
21   0.000028  0.000041  0.000064
22   0.000046  0.000041  0.000038
23   0.000020  0.000041  0.000026
24   0.000023  0.000081  0.000038
25   0.000026       NaN  0.000026
26   0.000017       NaN  0.000077
27   0.000026       NaN  0.000038
28   0.000040       NaN  0.000051
29   0.000017 

**在title对prefix的独立数上，三个数据集并没有太大差异_(:з」∠)_**

In [165]:
# 测试集中新prefix的平均样本数明显比训练集要高，进一步统计一下
print('-----------全部数据的统计特性：（顺序：5cvtrain，valid, test）-----------')
for tempDf in [df, validDf, predictDf]:
    print(tempDf[['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())

print('-----------只对新数据的统计特性: （顺序：5cvtrain，valid, test）-------------')
for tempDf in [df, validDf, predictDf]:
    print(tempDf[tempDf.prefix_newVal==1][['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())

-----------全部数据的统计特性：（顺序：5cvtrain，valid, test）-----------
       prefix_title_nunique  title_prefix_nunique  prefix_label_len  \
count          2.000000e+06          2.000000e+06      2.000000e+06   
mean           4.674695e+00          2.936306e+01      8.046599e+02   
std            2.245060e+00          1.130112e+02      2.317102e+03   
min            1.000000e+00          1.000000e+00      0.000000e+00   
25%            3.000000e+00          1.000000e+00      9.000000e+00   
50%            4.000000e+00          2.000000e+00      6.300000e+01   
75%            6.000000e+00          5.000000e+00      4.140000e+02   
max            1.400000e+01          1.089000e+03      1.760400e+04   

       title_label_len  
count     2.000000e+06  
mean      7.383130e+02  
std       2.500735e+03  
min       0.000000e+00  
25%       4.000000e+00  
50%       3.700000e+01  
75%       3.180000e+02  
max       1.969400e+04  
       prefix_title_nunique  title_prefix_nunique  prefix_label_len  \
count 

+ 训练集中新数据的prefix_title独立数只有1.3，但测试集中却高达3.3，为什么会这样？
+ 训练集中新数据的title_prefix独立数平均只有13，但测试集中，独立数依旧与整体数据集均值一样都是30，又是为什么？

In [148]:
# 数据集中有多少种prefix和title
for tempDf in [df, validDf, predictDf]:
    print('prefix种类:', tempDf['prefix'].drop_duplicates().shape[0], '平均样本数：', tempDf.shape[0] / tempDf['prefix'].drop_duplicates().shape[0])
    print('title种类:', tempDf['title'].drop_duplicates().shape[0], '平均样本数：', tempDf.shape[0] / tempDf['title'].drop_duplicates().shape[0])
    print('prefix/title种类比例:', tempDf['prefix'].drop_duplicates().shape[0] / tempDf['title'].drop_duplicates().shape[0])

prefix种类: 212171 平均样本数： 9.42635892746888
title种类: 350921 平均样本数： 5.699288443837787
prefix/title种类比例: 0.6046118642087536
prefix种类: 21694 平均样本数： 2.3047847331059277
title种类: 24688 平均样本数： 2.0252754374594946
prefix/title种类比例: 0.8787265068049255
prefix种类: 65018 平均样本数： 3.0760712418099603
title种类: 77925 平均样本数： 2.5665704202759065
prefix/title种类比例: 0.8343663779274944


**训练集数据基数大，平均样本数大很正常。但在prefix/title种类比上，训练集只有0.6，验证跟测试集都有0.8以上。训练集中涉及的title比例明显偏多**

In [31]:
# 方案1：假设将prefix分为两个池子抽样
prefixList = df['prefix'].drop_duplicates().values
np.random.shuffle(prefixList)
oldPrefix = prefixList[:int(len(prefixList)*0.8)]
newPrefix = prefixList[int(len(prefixList)*0.8):]

newDf = df[['prefix','query_prediction','title','tag','label']]
newDf = addGlobalFeas(newDf, statDf)
newDf = addHisFeas(newDf, newDf[(newDf.prefix.isin(oldPrefix))])

printInter(newDf)
print(newDf[['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())
print(newDf[newDf.prefix_newVal==1][['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())

prefix的新样本数: 415689 prefix的新样本率: 0.2078445
prefix的新种类数: 42435 新种类率: 0.200003770543571
title的新样本数: 191398 title的新样本率: 0.095699
title的新种类数: 62985 新种类率: 0.1794848413175615
只缺失prefix的样本数: 224291
只缺失title的样本数: 0
同时缺失prefix和title的样本数: 191398
       prefix_title_nunique  title_prefix_nunique  prefix_label_len  \
count          2.000000e+06          2.000000e+06      2.000000e+06   
mean           4.674695e+00          2.936306e+01      7.811492e+02   
std            2.245060e+00          1.130112e+02      2.669866e+03   
min            1.000000e+00          1.000000e+00      0.000000e+00   
25%            3.000000e+00          1.000000e+00      2.000000e+00   
50%            4.000000e+00          2.000000e+00      3.000000e+01   
75%            6.000000e+00          5.000000e+00      2.900000e+02   
max            1.400000e+01          1.089000e+03      2.195000e+04   

       title_label_len  
count     2.000000e+06  
mean      6.644143e+02  
std       1.973414e+03  
min       0.000000e+00  

In [22]:
# 方案2：将title相同的prefix分在一块抽样
prefixList = df.sort_values(['title'])['prefix'].drop_duplicates().values
oldPrefix = prefixList[:int(len(prefixList)*0.8)]
newPrefix = prefixList[int(len(prefixList)*0.8):]

newDf = df[['prefix','query_prediction','title','tag','label']]
newDf = addGlobalFeas(newDf, statDf)
newDf = addHisFeas(newDf, newDf[(newDf.prefix.isin(oldPrefix))])

printInter(newDf)
print(newDf[['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())
print(newDf[newDf.prefix_newVal==1][['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())

prefix的新样本数: 220600 prefix的新样本率: 0.1103
prefix的新种类数: 42435 新种类率: 0.200003770543571
title的新样本数: 170864 title的新样本率: 0.085432
title的新种类数: 59698 新种类率: 0.1701180607601141
只缺失prefix的样本数: 49736
只缺失title的样本数: 0
同时缺失prefix和title的样本数: 170864
       prefix_title_nunique  title_prefix_nunique  prefix_label_len  \
count         220600.000000         220600.000000          220600.0   
mean               3.396138              7.542203               0.0   
std                1.786942             53.540267               0.0   
min                1.000000              1.000000               0.0   
25%                2.000000              1.000000               0.0   
50%                3.000000              1.000000               0.0   
75%                4.000000              2.000000               0.0   
max               11.000000           1089.000000               0.0   

       title_label_len  prefix_title_label_len  
count    220600.000000                220600.0  
mean         44.958663        

### 基本思路
1. 测试集中，新prefix的平均样本数与旧prefix的平均样本数没有太大差距。而五折随机抽样中的新数据明显都是平均样本数较低的样本，推测在测试集中，相同prefix是整体抽样。
2. 最开始只对prefix进行分组抽样，得出的平均样本数很低，于是有了方案二，然而后来发现只是抽样不均匀的问题= =。重新打乱抽样后，得出的数据分布与方案二差不多。具体与方案二的差异需要进一步考究（现在脑子不够用），若无差异可以直接采用方案一

In [11]:
# 方案一封装(只按prefix抽取)
def newCvHisFea(df, nFold=5):
    kf = GroupKFold(n_splits=nFold)
    dfList = []
    for i, (statIdx, taskIdx) in enumerate(kf.split(df.values, df['label'].values, groups=df['prefix'].values)):
        tempDf = addHisFeas(df.iloc[taskIdx], df.iloc[statIdx])
        dfList.append(tempDf)
    df = pd.concat(dfList, ignore_index=True)
    return df

tempDf = df[['prefix','query_prediction','title','tag','label']]
tempDf = addGlobalFeas(tempDf, statDf)
tempDf = newCvHisFea(tempDf)

printInter(tempDf)
print(tempDf[tempDf.prefix_newVal==1][['prefix_title_nunique','title_prefix_nunique','prefix_label_len','title_label_len']].describe())

prefix的新样本数: 2000000 prefix的新样本率: 1.0
prefix的新种类数: 212171 新种类率: 1.0
title的新样本数: 936188 title的新样本率: 0.468094
title的新种类数: 314561 新种类率: 0.896386936091029
只缺失prefix的样本数: 1063812
只缺失title的样本数: 0
同时缺失prefix和title的样本数: 936188
       prefix_title_nunique  title_prefix_nunique  prefix_label_len  \
count          2.000000e+06          2.000000e+06         2000000.0   
mean           4.674695e+00          2.936306e+01               0.0   
std            2.245060e+00          1.130112e+02               0.0   
min            1.000000e+00          1.000000e+00               0.0   
25%            3.000000e+00          1.000000e+00               0.0   
50%            4.000000e+00          2.000000e+00               0.0   
75%            6.000000e+00          5.000000e+00               0.0   
max            1.400000e+01          1.089000e+03               0.0   

       title_label_len  
count     2.000000e+06  
mean      4.350884e+02  
std       1.932166e+03  
min       0.000000e+00  
25%       0.0000

In [8]:
# 新数据统计特性
def newDataStat(df):
    compDf = pd.DataFrame(columns=['all_data','new_data','old_data'])
    for x in ['prefix','title']:
        print('%s的新样本数:'%x, df['%s_newVal'%x].sum(), '%s的新样本率:'%x, df['%s_newVal'%x].mean())
        tempDf = df.drop_duplicates([x])
#         print('%s的新种类数:'%x, tempDf['%s_newVal'%x].sum(), '新种类率:', tempDf['%s_newVal'%x].sum() / tempDf.shape[0])
#         compDf.loc['%s种类数'%x] = [tempDf.shape[0], tempDf['%s_newVal'%x].sum(), tempDf.shape[0]-tempDf['%s_newVal'%x].sum()]
#         compDf.loc['%s平均样本量'%x] = [df.shape[0] / tempDf.shape[0], 
#                                    df['%s_newVal'%x].sum() / tempDf['%s_newVal'%x].sum()]
#         compDf.loc['%s平均样本量'%x] = [df['%s_label_len'%x].mean(), 
#                                    df[df['%s_newVal'%x]==1]['%s_label_len'%x].mean(),
#                                   df[df['%s_newVal'%x]==0]['%s_label_len'%x].mean()]
    print('只缺失prefix的样本数:', df[(df.prefix_newVal==1)&(df.title_newVal==0)].shape[0])
    print('只缺失title的样本数:', df[(df.prefix_newVal==0)&(df.title_newVal==1)].shape[0])
    print('同时缺失prefix和title的样本数:', df[(df.prefix_newVal==1)&(df.title_newVal==1)].shape[0])
#     compDf.loc['prefix/title种类比'] = [tempDf['prefix'].drop_duplicates().shape[0] / tempDf['title'].drop_duplicates().shape[0],
#                                 tempDf[df.prefix_newVal==1]['prefix'].drop_duplicates().shape[0] / tempDf[df.prefix_newVal==1]['title'].drop_duplicates().shape[0]]
    feaCol = [
        'label',
        'prefix_title_nunique','title_prefix_nunique',
        'prefix_label_len','title_label_len',
        'prefix_label_ratio','title_label_ratio',
        'prefix_title_label_len','prefix_title_label_ratio']
    for fea in feaCol:
        compDf.loc[fea, 'all_data'] = df[fea].mean()
        compDf.loc[fea, 'new_data'] = df[df.prefix_newVal==1][fea].mean()
        compDf.loc[fea, 'old_data'] = df[df.prefix_newVal==0][fea].mean()
    print(compDf)

In [16]:
# 整理对比
print('----------验证集样本特性-----------')
newDataStat(validDf)
print('----------测试集样本特性-----------')
newDataStat(predictDf)
print('----------训练集原5折交叉方案的样本特性-----------')
newDataStat(df)
print('----------训练集原5折不打乱交叉方案的样本特性-----------')
newDataStat(df2)
print('----------新方案只抽五分之一数据处理----------')
# 方案1：只处理一折
prefixList = df['prefix'].drop_duplicates().values
np.random.shuffle(prefixList)
oldPrefix = prefixList[:int(len(prefixList)*0.8)]
newPrefix = prefixList[int(len(prefixList)*0.8):]

tempDf = df[['prefix','query_prediction','title','tag','label']]
tempDf = addGlobalFeas(tempDf, statDf)
tempDf = addHisFeas(tempDf, tempDf[(tempDf.prefix.isin(oldPrefix))])
newDataStat(tempDf)
print('----------新方案五折交叉处理（上面只处理了1折，这里5折全处理）----------')
tempDf = df[['prefix','query_prediction','title','tag','label']]
tempDf = addGlobalFeas(tempDf, statDf)
tempDf = newCvHisFea(tempDf)
newDataStat(tempDf)

----------验证集样本特性-----------
prefix的新样本数: 18824 prefix的新样本率: 0.37648
prefix的新种类数: 5601 新种类率: 0.25818198580252605
title的新样本数: 8832 title的新样本率: 0.17664
title的新种类数: 6130 新种类率: 0.24829876863253403
只缺失prefix的样本数: 11402
只缺失title的样本数: 1410
同时缺失prefix和title的样本数: 7422
                      all_data  new_data
prefix种类数                21694      5601
prefix平均样本量            2.30478   3.36083
title种类数                 24688      6130
title平均样本量             2.02528   1.44078
prefix/title种类比       0.805493  0.772353
label                  0.37688  0.381215
prefix_title_nunique   4.20988   3.37218
title_prefix_nunique    30.572   32.2773
prefix_label_len       606.328         0
title_label_len        782.213   607.086
----------测试集样本特性-----------
prefix的新样本数: 82246 prefix的新样本率: 0.41123
prefix的新种类数: 23262 新种类率: 0.3577778461349165
title的新样本数: 40448 title的新样本率: 0.20224


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


title的新种类数: 26734 新种类率: 0.3430734680782804
只缺失prefix的样本数: 46942
只缺失title的样本数: 5144
同时缺失prefix和title的样本数: 35304
                      all_data  new_data
prefix种类数                65018     23262
prefix平均样本量            3.07607   3.53564
title种类数                 77925     26734
title平均样本量             2.56657   1.51298
prefix/title种类比       0.750106  0.739834
label                      NaN       NaN
prefix_title_nunique   4.12319   3.23878
title_prefix_nunique   29.0508   27.8598
prefix_label_len       612.609         0
title_label_len        717.963     373.8
----------训练集原5折交叉方案的样本特性-----------
prefix的新样本数: 97661 prefix的新样本率: 0.0488305
prefix的新种类数: 84481 新种类率: 0.3981741142757493
title的新样本数: 238352 title的新样本率: 0.119176
title的新种类数: 226218 新种类率: 0.6446408165940483
只缺失prefix的样本数: 18838
只缺失title的样本数: 159529
同时缺失prefix和title的样本数: 78823
                      all_data  new_data
prefix种类数               212171     84481
prefix平均样本量            9.42636   1.15601
title种类数                350921    2262

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


                      all_data  new_data
prefix种类数               212171    212171
prefix平均样本量            9.42636   9.42636
title种类数                350921    314561
title平均样本量             5.69929   2.97617
prefix/title种类比       0.544949  0.544949
label                 0.372959  0.372959
prefix_title_nunique   4.67469   4.67469
title_prefix_nunique   29.3631   29.3631
prefix_label_len             0         0
title_label_len        435.088   435.088


In [19]:
print('----------训练集原10折不打乱交叉方案的样本特性-----------')
newDataStat(df10)

----------训练集原10折不打乱交叉方案的样本特性-----------
prefix的新样本数: 926365 prefix的新样本率: 0.4631825
prefix的新种类数: 140268 新种类率: 0.6611082570191025
title的新样本数: 511216 title的新样本率: 0.255608
title的新种类数: 262135 新种类率: 0.7469914881127091
只缺失prefix的样本数: 500437
只缺失title的样本数: 85288
同时缺失prefix和title的样本数: 425928


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


                      all_data  new_data
prefix种类数               212171    140268
prefix平均样本量            9.42636   6.60425
title种类数                350921    262135
title平均样本量             5.69929    1.9502
prefix/title种类比       0.545083  0.630421
label                 0.372959  0.376004
prefix_title_nunique   4.67469   4.46459
title_prefix_nunique   29.3631   28.2085
prefix_label_len       685.808         0
title_label_len        753.864   359.001


## 样本权重方案研究

In [16]:
# 训练集五折、训练集+验证集五折跟验证集和测试集分布对比
# print('----------训练集打乱五折-----------')
# newDataStat(df)
print('----------训练集不打乱五折-----------')
newDataStat(newDf)
print('----------合并集不打乱五折-----------')
newDataStat(statDf)
print('----------验证集-----------')
newDataStat(validDf)
print('----------测试集-----------')
newDataStat(testDf)

----------训练集不打乱五折-----------
prefix的新样本数: 940520 prefix的新样本率: 0.47026
title的新样本数: 533323 title的新样本率: 0.2666615
只缺失prefix的样本数: 493298
只缺失title的样本数: 86101
同时缺失prefix和title的样本数: 447222
                          all_data  new_data  old_data
label                     0.372959  0.375941  0.370313
prefix_title_nunique       4.56732       NaN   4.56732
title_prefix_nunique       29.5853   38.5322   25.0512
prefix_label_len           607.453         0    1146.7
title_label_len            669.968   317.483   982.876
prefix_label_ratio        0.370172       NaN  0.370172
title_label_ratio         0.379545   0.39315   0.37265
prefix_title_label_len     246.609         0   465.528
prefix_title_label_ratio  0.372944       NaN  0.372944
----------合并集不打乱五折-----------
prefix的新样本数: 539600 prefix的新样本率: 0.26321951219512196
title的新样本数: 459486 title的新样本率: 0.22413951219512196
只缺失prefix的样本数: 217681
只缺失title的样本数: 137567
同时缺失prefix和title的样本数: 321919
                          all_data  new_data  old_data
label 

**合并后再五折的分布反而没有那么接近了**

In [18]:
# 训练集与验证集合并方案分布对比
print('----------train+valid合并五折---------')
tempDf = pd.concat([statDf] + [statDf.tail(50000)]*7, ignore_index=True)
newDataStat(tempDf)
print('----------测试集---------')
newDataStat(testDf)
print('----------train五折+valid---------')
tempDf = pd.concat([newDf] + [validDf.tail(50000)]*8, ignore_index=True)
newDataStat(tempDf)

----------train+valid合并五折---------
prefix的新样本数: 689190 prefix的新样本率: 0.2871625
title的新样本数: 533273 title的新样本率: 0.22219708333333332
只缺失prefix的样本数: 303620
只缺失title的样本数: 147703
同时缺失prefix和title的样本数: 385570
                          all_data  new_data  old_data
label                     0.373613  0.381925  0.370264
prefix_title_nunique       4.05197       NaN   4.05197
title_prefix_nunique       29.0311   37.7533   27.3369
prefix_label_len           592.001         0   830.486
title_label_len            677.898   219.247   862.663
prefix_label_ratio        0.368275       NaN  0.368275
title_label_ratio         0.377583  0.400167  0.373197
prefix_title_label_len     240.095         0   336.816
prefix_title_label_ratio   0.37528       NaN   0.37528
----------测试集---------
prefix的新样本数: 82406 prefix的新样本率: 0.41203
title的新样本数: 32491 title的新样本率: 0.162455
只缺失prefix的样本数: 55066
只缺失title的样本数: 5151
同时缺失prefix和title的样本数: 27340
                          all_data  new_data  old_data
label                   

## 新分布方案分析

In [9]:
# 构造数据集方案
def newCvHisFea(df, nFold=5, newRatio=0.4, random_state=0):
    kf = StratifiedKFold(n_splits=nFold, shuffle=True, random_state=random_state)
    dfList = []
    for i, (statIdx, taskIdx) in enumerate(kf.split(df.values, df['label'].values, groups=df['prefix'].values)):
        tempDf = addHisFeas(df.iloc[taskIdx], df.iloc[statIdx])
        tempDf.drop(tempDf[(tempDf.prefix_newVal==1)|(tempDf.title_newVal==1)].index, inplace=True)
        dfList.append(tempDf)
    oldDf = pd.concat(dfList, ignore_index=True)

    kf = GroupKFold(n_splits=nFold)
    dfList = []
    for i, (statIdx, taskIdx) in enumerate(kf.split(df.values, df['label'].values, groups=df['prefix'].values)):
        tempDf = addHisFeas(df.iloc[taskIdx], df.iloc[statIdx])
        dfList.append(tempDf)
    newDf = pd.concat(dfList, ignore_index=True)

    newSampleNum = int(len(oldDf)/(1-newRatio)*newRatio)
    df = pd.concat([oldDf, newDf.sample(n=newSampleNum, random_state=random_state)], ignore_index=True)
    return df

In [20]:
print(validDf.columns)

Index(['instance_id', 'prefix', 'title', 'tag', 'label', 'query_prediction',
       'query_predict_num', 'query_word', 'query_ratio', 'flag',
       'prefix_title_nunique', 'title_prefix_nunique', 'prefix_tag_nunique',
       'title_tag_nunique', 'prefix_newVal', 'title_newVal', 'tag_newVal',
       'prefix_label_len', 'prefix_label_sum', 'prefix_label_ratio',
       'title_label_len', 'title_label_sum', 'title_label_ratio',
       'tag_label_len', 'tag_label_sum', 'tag_label_ratio',
       'prefix_title_label_len', 'prefix_title_label_sum',
       'prefix_title_label_ratio', 'prefix_tag_label_len',
       'prefix_tag_label_sum', 'prefix_tag_label_ratio', 'title_tag_label_len',
       'title_tag_label_sum', 'title_tag_label_ratio'],
      dtype='object')


In [10]:
# 线下训练集方案对比
cols = ['instance_id', 'prefix', 'title', 'tag', 'label', 'query_prediction',
       'query_predict_num', 'query_word', 'query_ratio', 'flag']
newDf2 = newCvHisFea(df[cols])
print('----------训练集五折---------')
newDataStat(newDf)
print('----------验证集五折---------')
newDataStat(validDf)
print('----------构造训练集----------')
newDataStat(newDf2)

----------训练集五折---------
prefix的新样本数: 940520 prefix的新样本率: 0.47026
title的新样本数: 533323 title的新样本率: 0.2666615
只缺失prefix的样本数: 493298
只缺失title的样本数: 86101
同时缺失prefix和title的样本数: 447222
                          all_data  new_data  old_data
label                     0.372959  0.375941  0.370313
prefix_title_nunique       4.56732       NaN   4.56732
title_prefix_nunique       29.5853   38.5322   25.0512
prefix_label_len           607.453         0    1146.7
title_label_len            669.968   317.483   982.876
prefix_label_ratio        0.370172       NaN  0.370172
title_label_ratio         0.379545   0.39315   0.37265
prefix_title_label_len     246.609         0   465.528
prefix_title_label_ratio  0.372944       NaN  0.372944
----------验证集五折---------
prefix的新样本数: 18824 prefix的新样本率: 0.37648
title的新样本数: 8832 title的新样本率: 0.17664
只缺失prefix的样本数: 11402
只缺失title的样本数: 1410
同时缺失prefix和title的样本数: 7422
                          all_data  new_data  old_data
label                      0.37688  0.381215  0.

In [15]:
# 合并的线上训练方案对比
cols = ['instance_id', 'prefix', 'title', 'tag', 'label', 'query_prediction',
       'query_predict_num', 'query_word', 'query_ratio', 'flag']
statDf2 = newCvHisFea(pd.concat([df[cols],validDf[cols]], ignore_index=True))
print('----------合并集五折---------')
newDataStat(statDf)
print('----------测试集五折---------')
newDataStat(testDf)
print('----------构造合并集----------')
newDataStat(statDf2)

----------合并集五折---------
prefix的新样本数: 539600 prefix的新样本率: 0.26321951219512196
title的新样本数: 459486 title的新样本率: 0.22413951219512196
只缺失prefix的样本数: 217681
只缺失title的样本数: 137567
同时缺失prefix和title的样本数: 321919
                          all_data  new_data  old_data
label                     0.373055  0.382402  0.369716
prefix_title_nunique       3.98389       NaN   3.98389
title_prefix_nunique       29.0265    36.806   27.7929
prefix_label_len           606.291         0   822.893
title_label_len            681.676   148.542   872.141
prefix_label_ratio        0.367796       NaN  0.367796
title_label_ratio         0.377189  0.405032  0.372774
prefix_title_label_len     247.216         0   335.535
prefix_title_label_ratio  0.375035       NaN  0.375035
----------测试集五折---------
prefix的新样本数: 82406 prefix的新样本率: 0.41203
title的新样本数: 32491 title的新样本率: 0.162455
只缺失prefix的样本数: 55066
只缺失title的样本数: 5151
同时缺失prefix和title的样本数: 27340
                          all_data  new_data  old_data
label                 

In [ ]:
# 不合并的线上训练方案对比
print('----------训练集五折+valid---------')
tempDf = pd.concat([newDf, validDf], ignore_index=True)
newDataStat(tempDf)
print('----------测试集五折---------')
newDataStat(testDf)
print('----------构造训练集+valid----------')
tempDf = pd.concat([newDf2, validDf], ignore_index=True)
newDataStat(tempDf)

----------训练集五折+valid---------
prefix的新样本数: 959344 prefix的新样本率: 0.46797268292682925
title的新样本数: 542155 title的新样本率: 0.2644658536585366
只缺失prefix的样本数: 504700
只缺失title的样本数: 87511
同时缺失prefix和title的样本数: 454644
                          all_data  new_data  old_data
label                     0.373055  0.376044  0.370426
prefix_title_nunique       4.56906       NaN   4.56906
title_prefix_nunique       29.6626   38.6896    25.121
prefix_label_len           607.425         0   1141.72
title_label_len            672.706   323.165   980.163
prefix_label_ratio        0.370224       NaN  0.370224
title_label_ratio          0.37953  0.393079  0.372713
prefix_title_label_len     246.367         0   463.071
prefix_title_label_ratio  0.373027       NaN  0.373027
----------测试集五折---------
prefix的新样本数: 82406 prefix的新样本率: 0.41203
title的新样本数: 32491 title的新样本率: 0.162455
只缺失prefix的样本数: 55066
只缺失title的样本数: 5151
同时缺失prefix和title的样本数: 27340
                          all_data  new_data  old_data
label             

In [65]:
# 剔除掉五折交叉中的新数据
tempDf = df[['prefix','query_prediction','title','tag','label']]
tempDf = addGlobalFeas(tempDf, statDf)
tempDf = newCvHisFea2(tempDf)
newDataStat(tempDf)

prefix的新样本数: 1162183 prefix的新样本率: 0.3999999311640368
prefix的新种类数: 84551 新种类率: 0.4851974911196424
title的新样本数: 543903 title的新样本率: 0.1872004344925998
title的新种类数: 132835 新种类率: 0.5129952884838186
只缺失prefix的样本数: 618280
只缺失title的样本数: 0
同时缺失prefix和title的样本数: 543903


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


                      all_data  new_data
prefix种类数               174261     84551
prefix平均样本量             16.673   13.7453
title种类数                258940    132835
title平均样本量             11.2206   4.09458
prefix/title种类比       0.589774  0.799006
label                 0.372703  0.372778
prefix_title_nunique   4.88597   4.67228
title_prefix_nunique   31.4733   29.4519
prefix_label_len       553.616         0
title_label_len        680.158   434.567
